# FINAL PROJECT: HARMONIC COORDINATES FOR SPATIAL MESH EDITING


In [17]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg
import igl
import triangle as tr
import ipywidgets as iw
import time
import meshplot as mp

class HarmonicDeformer:
    #class to interactively move a ed object by moving a boundary cage!
    #we precompute harmonic weights of the cage points before defomrs
    #apply new cage points to defomr
    def __init__(self, cage_v, cage_f, cage_before_triangulation):
        """
        Parameters:
        - cage_v: array of triangulated cage vertex positions (boundary + interior)
        - cage_f: array of triangle face indices to cage v
        - cage_before_triangulation: (K, 2) original cage (before triangulation)

        Storing: 
        - cage handles (boundary vs)
        - number of handle vertices
        - harmonic_basis: to once computed store harmonic weights
        """
        self.cage_v = cage_v
        self.cage_f = cage_f
        self.cage_handle_indices = np.arange(cage_before_triangulation.shape[0])
        
        self.num_vertices = cage_v.shape[0]
        self.cage_indicis_total= np.arange(self.num_vertices)
        
        
        self.num_boundary = len(self.cage_handle_indices)
        self.harmonic_basis = None 

    def precompute_weights(self):
        '''
        COMPUTES THE HARMONIC BASIS FUNCTION for the cage

        We want to solve a Laplace equation: Δphi= 0 over the cage where phi is a scalar function. 
        This is the harmonic functions and we discretize them into a linear system using: 
        - L : cotan weight laplacian matrix 
        - M: mass matrix

        we then solve A = M_inv @ L --> A @ (PHI_MATRIX) = 0

        - partition A into constrained (cage boundary) points and free (interior cage) points

        Boils down to : Afc @ phi_constrained + Aff @ phi_free =0
        FINALLY HARMONIC BASIS: 

        solve for phi_free!  Aff @ phi_f = - Afc @ phi_c  
                                   |----|
                                     x 
        Then stack phi_constrained and phi_interior to get the Harmonic basis for the cage points. 
        '''
        boundary_idx = self.cage_handle_indices
        interior_idx = np.setdiff1d(self.cage_indicis_total, boundary_idx)
        
        L = igl.cotmatrix(self.cage_v, self.cage_f)
        M = igl.massmatrix(self.cage_v, self.cage_f, igl.MASSMATRIX_TYPE_VORONOI)
        Minv = sp.diags(1.0 / M.diagonal())
        A = Minv @ L

        Aff = A[interior_idx, :][:, interior_idx]
        Afc = A[interior_idx, :][:, boundary_idx]

        phi_c = np.eye(self.num_boundary)
        rhs = -Afc @ phi_c
        phi_f = sp.linalg.spsolve(Aff, rhs)


        PHI = np.zeros((self.num_vertices, self.num_boundary))
        PHI[boundary_idx, :] = phi_c
        PHI[interior_idx, :] = phi_f

        self.harmonic_basis = PHI
        '''HARMONIC BASIS IS SIZE: VERTICES BY BOUNDARY VERTICES: 
        - EACH ROW CORRESPONDS TO A VERTEX IN THE TRIANGULATED CAGE
        - EACH COLUMN CORRECPONDS TO HARMONIC BASIS FUNCTION 
                - 1 AT BOUNDARY Vi  
                - 0 AT ALL OTHER BOUNDARY Vs 
                - SMOOTH ELSEWHERE           '''

        print(" Harmonic basis computed. Shape:", PHI.shape)

    def deform(self, new_cage_positions):
        '''
        Given new positions for cage boundary vertices the function computes 
        DEFORMED POSITIONS of all cage vertices (boundary and interior vertices)  
        based on precomputed harmonic basis
        '''
        return self.harmonic_basis @ new_cage_positions


    def compute_weights_for_mesh(self, object_vertices):
        '''
        Compute HARMONIC COORDINATES for the set of object vertices 
        - HOW EACH OBJECT VERTEX can be expressed as a combination of boundary cage vertices
          using BARYCENTRIC INTERPOLATION over the harmonic basis function
        '''
        num_obj_v = object_vertices.shape[0]
        num_basis = self.harmonic_basis.shape[1]
        print('num of basis' , num_basis, 'and num of obj_v',num_obj_v )

        #weights: each row i holds HARMONIC COORD VECTOR FOR OBJECT POINT I (pi)
        weights = np.zeros((num_obj_v, num_basis))
        

        for i in range(num_obj_v): 
            #for each mesh vertex: 
        
            p = object_vertices[i]
            #express the mesh vertex by a barycentric coordinate of a cage triangle. 
            
            #loop through all the CAGE TRIANGLED 
            #to find th etriangle containing point p 
            for tri in self.cage_f:
                
                #a,b,c are the vertices in that face 
                a= self.cage_v[tri[0], :2]
                b= self.cage_v[tri[1], :2]
                c= self.cage_v[tri[2], :2]
                
                #if p is in triangle abc barycentric coords returned
                bary = barycentric_coordinates(p, a, b, c)
                
                if bary is not None:
                    #baricentric coordinates found : 
                    #INTERPOLATE harmonic coordinate vector from:
                        #harmonic basis
                        #barycentric coefficients
                    
                    u, v, w = bary
                    phi_a = self.harmonic_basis[tri[0], :]
                    phi_b = self.harmonic_basis[tri[1], :]
                    phi_c = self.harmonic_basis[tri[2], :]
                    
                    #HARMONIC COORD FOR MESHPOINT Pi
                    weights[i, :] = u * phi_a + v * phi_b + w * phi_c
                    
                    break
        return weights


"""HELPER FUNCTIONS"""

def barycentric_coordinates(p, a, b, c):
    '''
    we use barycentric coordinates u,v,w to interpolate harmonic basis values 
    fro the triangles three vertices to point p 

    each mesh point p gets a harmonic coordinate vector (weights for each cage handle)
    '''
    def area(p1, p2, p3):
        return 0.5 * np.linalg.det(np.array([
            [p2[0] - p1[0], p2[1] - p1[1]],
            [p3[0] - p1[0], p3[1] - p1[1]]
        ]))

    total_area = area(a, b, c)
    
    if np.isclose(total_area, 0.0):
        return None

    u = area(p, b, c)/ total_area
    v = area(p, c, a)/ total_area
    w = area(p, a, b)/ total_area

    if u< 0 or v< 0 or w< 0:
        return None

    return np.array([u, v, w])

def build_edges_loop(coords):
    n = len(coords)
    return np.array([[i, (i + 1) % n] for i in range(n)])


### Step 1: from my character make a cage and triangulate the cage
- this triangulation is necessary to later distribute the weight eachh cage point has on the points of the object mesh.



In [18]:
v, f = igl.read_triangle_mesh("data/woody-hi.off")
v_2d = v[:, :2]
v_2d -= v_2d.min(axis=0)
v_2d /= v_2d.max()

cage = np.load("data/woody-hi.cage.npy")
if np.allclose(cage[0], cage[-1]):
    cage = cage[:-1]  # Ensure open loop

#TRIANGULATE CAGE:
cage_dict = {"vertices": cage, "segments": build_edges_loop(cage)}
cage_tri = tr.triangulate(cage_dict, 'pq30a0.01')  

cage_v = np.hstack((cage_tri['vertices'], np.zeros((len(cage_tri['vertices']), 1))))
cage_f = cage_tri['triangles']

mp.plot(cage_v, cage_f, shading={"wireframe": True, "line_color": "black"})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4304474…

### Step 2: Compute the harmonic weights 
- using the Harmonic Deformer class we compute the harmonic weights of cage points to the interior poiints of the cage.
- These weights will dictate how the change in a cage point will modeify the point in the mesh


In [19]:

# initialize harmonic deformer with full triangulated cage
deformer = HarmonicDeformer(cage_v, cage_f, cage)
deformer.precompute_weights()

# compute weight matrix for object vertices (harmonic coords)
hc = deformer.compute_weights_for_mesh(v_2d)



 Harmonic basis computed. Shape: (81, 27)
num of basis 27 and num of obj_v 2642


## Visualize

In [20]:
import numpy as np
import ipywidgets as iw
import meshplot as mp
import time


cage_v_copy = cage_v.copy()
v_copy = v.copy()
num_cage_boundary_v = len(cage)

p = mp.plot(v_copy, f, return_plot=True)
edges = np.array([[i, (i + 1) % num_cage_boundary_v] for i in range(num_cage_boundary_v)])
point_oid = p.add_points(cage, shading={"point_color": "green", "point_size": 0.1})
line_oid = p.add_lines(cage[edges[:, 0]], cage[edges[:, 1]])


def position_deformer(updated_cage_boundary):
    return hc @ updated_cage_boundary


def pos_f(selected_vertices, x, y, z):
    global point_oid, line_oid
    t0 = time.time()


    cage_v_copy[:] = cage_v
    offset = np.array([x, y, z])
    cage_v_copy[selected_vertices] += offset

    new_cage_boundary = cage_v_copy[:num_cage_boundary_v]
    v_deformed = pos_f.deformer(new_cage_boundary)

    p.remove_object(point_oid)
    p.remove_object(line_oid)
    color = np.zeros(num_cage_boundary_v)
    color[selected_vertices] = 1
    point_oid = p.add_points(new_cage_boundary, c=color, shading={"point_size": 0.1})
    line_oid = p.add_lines(new_cage_boundary[edges[:, 0]], new_cage_boundary[edges[:, 1]])

    p.update_object(oid=0, vertices=v_deformed)
    print("FPS:", 1 / (time.time() - t0))

pos_f.deformer = position_deformer


def widgets_wrapper():
    select = iw.SelectMultiple(
        options=np.arange(num_cage_boundary_v),
        rows=10,
        description="Cage Vertices"
    )
    sliders = {a: iw.FloatSlider(min=-1, max=1, step=0.01, value=0, description=a) for a in "xyz"}
    return dict(selected_vertices=select, **sliders)


iw.interact(pos_f, **widgets_wrapper())


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(SelectMultiple(description='Cage Vertices', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1…

<function __main__.pos_f(selected_vertices, x, y, z)>